In [1]:
# Imports
import pandas as pd
import plotly.express as px
import kaleido
import os
import scipy.stats as st
import math

### Utils

In [2]:
# Save image as file
def save_image(fig, filename):
    if not os.path.exists("images"):
        os.mkdir("images")
    
    fig.write_image("images/" + filename + ".svg")
    fig.write_image("images/" + filename + ".png")

In [14]:
# Get param dataframe given folder location and parameter name(s)
def get_params(folder, params):
    df = pd.read_csv(folder + "/params.csv", usecols=params)
    df.columns = df.columns.astype(str)

    return df

def get_results(folder, names):
    df = pd.read_csv(folder + "/results.csv", names=names)
    df.columns = df.columns.astype(str)
    
    return df

def calc_rpd(df):    
    columns = list(df.columns.values)
    columns = [str(elem) for elem in columns]
    for col in columns:
        df[col + "_rpd"] = (df[col] / df[columns].min(axis=1) - 1)*100
    
    return df
        
def ci_row(row, n):
    # ci = st.t.interval(alpha=0.95, df=n-1, loc=row["mean"], scale=row["e"])
    ci = st.norm.interval(0.95, loc=row["mean"], scale=row["e"]/math.sqrt(n))
    print(ci)
    return ci
        
def calc_stats(df):
    n = len(df)
    df = df.transpose()
    df["mean"] = df.mean(axis=1)
    df["sem"] = df.drop("mean", axis=1).apply(lambda x: x.sem(), axis=1)
    return df.filter(like="_rpd", axis=0)[["mean", "sem"]]

def plot_from_df(df, xname, yname, filename, new_names={}, rename=True, save=True):
    if rename:
        df.rename(index=lambda s: s.replace("_rpd", ""), inplace=True)
        
    if len(new_names) > 0:
        df.rename(index=new_names, inplace=True)
        
    fig = px.scatter(df, 
                     x=df.index, y="mean", 
                     width=500,
                     height=400,
                     template="simple_white",
                     error_y="sem")
    
    fig.update_layout(
        xaxis_type='category',
        xaxis_title=xname,
        yaxis_title=yname,
        font_family="Times New Roman"
    )

    fig.update_xaxes(mirror=True)
    fig.update_yaxes(mirror=True)

    fig.update_traces(marker_size=10)
    
    if save:
        save_image(fig, filename)
    
    return fig

# Create a plot from two different problem folders
def merge_and_plot(folders, names, x_title, y_title, filename="1_generational"):
    dfs = []
    for fname, colname in zip(folders, names):
        dfs.append(get_results(fname, [colname]))
        
    res = pd.concat(dfs, axis=1)
    
    rpd = calc_rpd(res)
    
    stats = calc_stats(rpd)

    fig = plot_from_df(stats, x_title, y_title, filename)
    
    return fig

# Create plot from single file, given a list of parameter names
def single_file_plot(
    folder, 
    param_names, 
    x_title, 
    y_title,
    filename,
    new_names={}, 
    print_first_row=False
):
    # Get the parameter values
    params = get_params(folder, param_names)
    
    params.index = params[param_names[0]].astype(str)
    if len(param_names) > 1:
        for val in param_names[1:]:
            params.index += ", " + params[val].astype(str)

    header = list(params.index.values)

    # Get the actual results
    res = get_results(folder, header)
    
    if print_first_row:
        print(res.head(5))

    rpd = calc_rpd(res)

    stats = calc_stats(rpd)
    return plot_from_df(stats, x_title, y_title, filename, new_names=new_names)

### 1. Generational scheme

In [15]:
folders = ["../solutions/generational", "../solutions/steady_state"]

fig = merge_and_plot(
    folders, 
    ["Generational", "Steady"], 
    r"$\text{Generational Scheme}$", 
    r"$\text{RPD(%)}$"
)

fig.update_layout(
    xaxis_range=[-0.5,1.5]
)

fig.show()

### Initialisation

In [5]:
new_names = {"0.2": "GCH (0.2)", "0.5": "GCH (0.5)", "1.0": "GCH (1.0)"}

fig = single_file_plot(
    "../solutions/initialisation", 
    ["construction"], 
    r"$\text{Initialisation}$", 
    r"$\text{RPD(%)}$", 
    "2_initialisation",
    new_names
)


fig.show()

### Population size

In [6]:
fig = single_file_plot(
    "../solutions/popsize", 
    ["pop_size"], 
    r"$\text{Population Size}$", 
    r"$\text{RPD(%)}$",
    "3_popsize",
)
fig.show()

### Tournament size

In [7]:
fig = single_file_plot(
    "../solutions/k_tournament", 
    ["k_tournament"], 
    r"$\text{Tournament Size}$", 
    r"$\text{RPD(%)}$", 
    "4_k_tournament"
)

fig.show()

### Crossover

In [8]:
fig = single_file_plot(
    "../solutions/xover", 
    ["xover_type"], 
    r"$\text{Crossover}$", 
    r"$\text{RPD(%)}$", 
    "5_crossover"
)

fig.show()

### Mutation

In [9]:
fig = single_file_plot(
    "../solutions/mutation", 
    ["mutation_type"], 
    r"$\text{Mutation type}$", 
    r"$\text{RPD(%)}$", 
    "6_mutation"
)

fig.show()

### Mutation probabilities

In [10]:
fig = single_file_plot(
    "../solutions/mutation-prob", 
    ["mutation_prob"], 
    r"$P_{m}$", 
    r"$\text{RPD(%)}$", 
    "7_mutation-prob"
)

fig.show()

### Local search

In [11]:
fig = single_file_plot(
    "../solutions/local_search", 
    ["approx_calc"], 
    r"$\text{Iterations}$", 
    r"$\text{RPD(%)}$", 
    "8_local-search"
)

fig.show()

### Crowding

In [12]:
fig = single_file_plot(
    "../solutions/crowding-deviation", 
    ["crowding_scale", "k_nearest"], 
    r"$\text{Crowding scale, k_nearest}$", 
    r"$\text{RPD(%)}$", 
    "9_crowding"
)

fig.update_layout(
        width=600
    )


fig.show()

### Replacement

In [18]:
fig = single_file_plot(
    "../solutions/replacement", 
    ["non_improving_iterations", "allways_keep"], 
    r"$\text{Iterations, Keep}$", 
    r"$\text{RPD(%)}$", 
    "10_replacement"
)

fig.update_layout(
        width=600
    )


fig.show()

### Iterated Greedy

In [13]:
fig = single_file_plot(
    "../solutions/ig", 
    ["t", "d"], 
    r"$\text{d, T}$", 
    r"$\text{RPD(%)}$", 
    "10_iterated-greedy"
)

fig.update_layout(
        width=600
    )


fig.show()